# ibm_db.fetch_assoc()

## Purpose:

Retrieve a row from a result set and copy its data to a dictionary.

## Syntax:

`dictionary ibm_db.fetch_assoc( IBM_DBStatement `*`statement`*` [, int `*`rowNumber`*`] )`

## Parameters:

* __*statement*__&nbsp; &nbsp; &nbsp; &nbsp; : A valid `IBM_DBStatement` object that contains a result set. 
* __rowNumber__&nbsp; &nbsp; &nbsp;: A specific, one-indexed, row in the result set that data is to be retrieved from.  

## Return values:

When a __*rowNumber*__ value <u>is</u> provided:
* If __successful__, a dictionary containing values for every column in the result set, *for the row specified*.
* If __unsuccessful__ or if the row specified doesn't exist, the value `False`.

When a __*rowNumber*__ value <u>is not</u> provided:   
* If __successful__, a dictionary containing values for every column in the result set, *for the next row*.
* If __unsuccessful__ or if there are no rows left in the result set, the value `False`.

## Description:

The __ibm_db.fetch_assoc()__ API is used to retrieve a row (record) from a result set and copy its data to a dictionary. Depending on how it is called, it can advance a cursor to the next row in a result set and copy the data for that row into a dictionary. Or, it can be used to retrieve the data for a specific row  &mdash; provided a keyset-driven, dynamic, or static cursor is used to traverse the result set. (*A warning will be issued if a specific row is requested and a forward-only cursor is used*.)<p>
    
In either case, the value for the first column in the row will be stored in the first key-value pair of the dictionary &mdash; the key will be the name of the column; the value will be the data for that column in the row. The second column will be stored in the second key-value pair, and so on.

## Example:

In [1]:
#----------------------------------------------------------------------------------------------#
#  NAME:     ibm_db-fetch_assoc.py                                                             #
#                                                                                              #
#  PURPOSE:  This program is designed to illustrate how to use the ibm_db.fetch_assoc() API.   #
#                                                                                              #
#            Additional APIs used:                                                             #
#                 ibm_db.exec_immediate()                                                      #
#                                                                                              #
#----------------------------------------------------------------------------------------------#
#                     DISCLAIMER OF WARRANTIES AND LIMITATION OF LIABILITY                     #
#                                                                                              #
#  (C) COPYRIGHT International Business Machines Corp. 2018, 2019 All Rights Reserved          #
#  Licensed Materials - Property of IBM                                                        #
#                                                                                              #
#  US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA    #
#  ADP Schedule Contract with IBM Corp.                                                        #
#                                                                                              #
#  The following source code ("Sample") is owned by International Business Machines            #
#  Corporation ("IBM") or one of its subsidiaries and is copyrighted and licensed, not sold.   #
#  You may use, copy, modify, and distribute the Sample in any form without payment to IBM,    #
#  for the purpose of assisting you in the creation of Python applications using the ibm_db    #
#  library.                                                                                    #
#                                                                                              #
#  The Sample code is provided to you on an "AS IS" basis, without warranty of any kind. IBM   #
#  HEREBY EXPRESSLY DISCLAIMS ALL WARRANTIES, EITHER EXPRESS OR IMPLIED, INCLUDING, BUT NOT    #
#  LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE. #
#  Some jurisdictions do not allow for the exclusion or limitation of implied warranties, so   #
#  the above limitations or exclusions may not apply to you. IBM shall not be liable for any   #
#  damages you suffer as a result of using, copying, modifying or distributing the Sample,     #
#  even if IBM has been advised of the possibility of such damages.                            #
#----------------------------------------------------------------------------------------------#

# Load The Appropriate Python Modules
import sys         # Provides Information About Python Interpreter Constants And Functions
import ibm_db      # Contains The APIs Needed To Work With Db2 Databases

#----------------------------------------------------------------------------------------------#
# Import The Db2ConnectionMgr Class Definition, Attributes, And Methods That Have Been Defined #
# In The File Named "ibm_db_tools.py"; This Class Contains The Programming Logic Needed To     #
# Establish And Terminate A Connection To A Db2 Server Or Database                             #
#----------------------------------------------------------------------------------------------#
from ibm_db_tools import Db2ConnectionMgr

#----------------------------------------------------------------------------------------------#
# Import The ipynb_exit Class Definition, Attributes, And Methods That Have Been Defined In    #
# The File Named "ipynb_exit.py"; This Class Contains The Programming Logic Needed To Allow    #
# "exit()" Functionality To Work Without Raising An Error Or Stopping The Kernel If The        #
# Application Is Invoked In A Jupyter Notebook                                                 #
#----------------------------------------------------------------------------------------------#
from ipynb_exit import exit

# Define And Initialize The Appropriate Variables
dbName = "SAMPLE"
userID = "db2inst1"
passWord = "Passw0rd"
dbConnection = None
resultSet = False
dataRecord = False

# Create An Instance Of The Db2ConnectionMgr Class And Use It To Connect To A Db2 Database
conn = Db2ConnectionMgr('DB', dbName, '', '', userID, passWord)
conn.openConnection()
if conn.returnCode is True:
    dbConnection = conn.connectionID
else:
    conn.closeConnection()
    exit(-1)

# Define The SQL Statement That Is To Be Executed
sqlStatement = "SELECT deptname FROM department WHERE admrdept = 'A00'"

# Execute The SQL Statement Just Defined
print("Executing the SQL statement \"" + sqlStatement + "\" ... ", end="")
try:
    resultSet = ibm_db.exec_immediate(dbConnection, sqlStatement)
except Exception:
    pass

# If The SQL Statement Could Not Be Executed, Display An Error Message And Exit 
if resultSet is False:
    print("\nERROR: Unable to execute the SQL statement specified.\n")
    conn.closeConnection()
    exit(-1)

# Otherwise, Complete The Status Message
else:
    print("Done!\n")

# Display A Report Header
print("Query results:\n")
print("DEPTNAME")
print("____________________________")

# As Long As There Are Records In The Result Set Produced, ...
noData = False
while noData is False:

    # Retrieve A Record And Store It In A Python Dictionary
    try:
        dataRecord = ibm_db.fetch_assoc(resultSet)
    except:
        pass

    # If The Data Could Not Be Retrieved Or There Was No Data To Retrieve, Set The
    # "No Data" Flag And Exit The Loop 
    if dataRecord is False:
        noData = True

    # Otherwise, Display The Data Retrieved
    else:
        print("{:20}" .format(dataRecord['DEPTNAME']))

# Add A Blank Line To The End Of The Report
print()

# Close The Database Connection That Was Opened Earlier
conn.closeConnection()

# Return Control To The Operating System
exit()


Connecting to the SAMPLE database ... Done!

Executing the SQL statement "SELECT deptname FROM department WHERE admrdept = 'A00'" ... Done!

Query results:

DEPTNAME
____________________________
SPIFFY COMPUTER SERVICE DIV.
PLANNING            
INFORMATION CENTER  
DEVELOPMENT CENTER  
SUPPORT SERVICES    

Disconnecting from the SAMPLE database ... Done!

